In [3]:
%pip install sqlalchemy
%pip install pymysql

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.1 MB 1.3 MB/s eta 0:00:02
   ---------------------------------------  2.1/2.1 MB 19.3 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 16.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/297.0 kB ? eta -:--:--
   --------------------------------------- 297.0/297.0 kB 17.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\bbizi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   --------- ------------------------------ 10.2/45.0 kB ? eta -:--:--
   ------------------ --------------------- 20.5/45.0 kB 320.0 kB/s eta 0:00:01
   ---------------------------------------- 45.0/45.0 kB 315.7 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\bbizi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import requests
import json
import time
import re
import pandas as pd
from sqlalchemy import create_engine


In [5]:
#rect: 지도 영역 좌표(검색 범위)
#theme_id: 음식점이면 'c9'

jsonp = requests.get('https://search.map.kakao.com/mapsearch/theme.daum?output=json&category=Y&callback=jQuery18109872917320938157_1753103241831&theme_id=c9&cpage=1&rect=487005%2C1116407%2C493715%2C1122797',
                   headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
                            'referer':'https://map.kakao.com/',
                            })

jsonp.status_code

200

In [6]:
#jsonp 형식으로 반환 되는걸 알수 있다.
jsonp.text

'/**/jQuery18109872917320938157_1753103241831({"place_totalcount":2383,"page_count":500,"sort":0,"theme_id":"c9","page":1,"place":[{"confirmid":"20526340","x":487598,"y":1122355,"lon":126.9438663,"lat":37.53997206,"name":"원조조박집 본관","tel":"02-712-7462","address":"서울 마포구 용강동 40-1 1층","reviewCount":317,"homepage":"https://www.instagram.com/jobakzib_official","img":"http://t1.kakaocdn.net/mystore/8A76F8C79C0B4BA0A97F01ADEAFF4825","sourceId":"","source":"","roadview":"1197590398|133|1|||487569|1122382|1","full_category_ids":"9|124|18165","last_cate_id":"18165","last_cate_name":"육류,고기","last_cate_depth":"3","cate_name_depth1":"음식점","cate_name_depth2":"한식","cate_name_depth3":"육류,고기","cate_name_depth4":"","cate_name_depth5":"","hub_data":"","brand":"","brandName":"육류,고기","oil1":"0","oil2":"0","oil3":"0","oil4":"0","oilTime":"","oil_sel24":"","phoneSynonyms":"","related_place":"","new_address":"서울 마포구 토정로 313-1","courseinfo":"","geoinfo":"","requiringtime":"","tvshow_info":"","tvshow_name":"","

In [7]:
#정규 표현식으로 json 추출하기.
res = re.search(r'jQuery\d+_\d+\((.*)\)', jsonp.text)

In [8]:
json_str = res.group(1)

data = json.loads(json_str)

data

{'place_totalcount': 2383,
 'page_count': 500,
 'sort': 0,
 'theme_id': 'c9',
 'page': 1,
 'place': [{'confirmid': '20526340',
   'x': 487598,
   'y': 1122355,
   'lon': 126.9438663,
   'lat': 37.53997206,
   'name': '원조조박집 본관',
   'tel': '02-712-7462',
   'address': '서울 마포구 용강동 40-1 1층',
   'reviewCount': 317,
   'homepage': 'https://www.instagram.com/jobakzib_official',
   'img': 'http://t1.kakaocdn.net/mystore/8A76F8C79C0B4BA0A97F01ADEAFF4825',
   'sourceId': '',
   'source': '',
   'roadview': '1197590398|133|1|||487569|1122382|1',
   'full_category_ids': '9|124|18165',
   'last_cate_id': '18165',
   'last_cate_name': '육류,고기',
   'last_cate_depth': '3',
   'cate_name_depth1': '음식점',
   'cate_name_depth2': '한식',
   'cate_name_depth3': '육류,고기',
   'cate_name_depth4': '',
   'cate_name_depth5': '',
   'hub_data': '',
   'brand': '',
   'brandName': '육류,고기',
   'oil1': '0',
   'oil2': '0',
   'oil3': '0',
   'oil4': '0',
   'oilTime': '',
   'oil_sel24': '',
   'phoneSynonyms': '',
   

In [9]:
#theme_id -테마 코드 ( 현재는 음식점 고정이나, 나중에 확장을 할수도 있어서 따로 파라미터로 뺌)
#rect- 지도 범위
def get_kakao_restaurants(theme_id,rect,max_page=20):
  all_places = []
  for cpage in range(1, max_page + 1 ):
    url = 'https://search.map.kakao.com/mapsearch/theme.daum'

    params = {
      'output':'json',
      'category':'y',
      'callback':'jQuery18109872917320938157_1753103241831',
      'theme_id':theme_id,
      'cpage':cpage,
      'rect':rect
    }

    headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36','referer':'https://map.kakao.com/'}

    jsonp = requests.get(url,params=params,headers=headers)
    if jsonp.status_code != 200:
      print(f'{cpage}페이지에서 에러가 발생했습니다.')
      break
    #앞에거 다 빼고 jQuery로 시작되는 부분부터 안에꺼 가져오기.
    res = re.search(r'jQuery\d+_\d+\((.*)\)', jsonp.text)

    if not res:
      print(f'{cpage}페이지에서 json 파싱 실패')

    #group(0) : 정규식 전체와 매치된 부분
    #group(1) : 첫 번째 소괄호 ( ... )에 해당하는 부분
    json_data = json.loads(res.group(1))
    places = json_data.get('place',[])

    if not places:
      break
    for p in places:
      all_places.append({
        'place_id':p['confirmid'],
        'name':p['name'],
        'address':p.get('address',''),
        'new_address':p.get('new_address',''),
        'lon':p.get('lon'),
        'lat':p.get('lat'),
        'tel':p.get('tel'),
        'category_depth1':p.get('cate_name_depth1'),
        'category_depth2':p.get('cate_name_depth2'),
        'category_depth3':p.get('cate_name_depth3'),
        'category_depth4':p.get('cate_name_depth4'),
        'category_depth5':p.get('cate_name_depth5'),
        'rating_average':p.get('rating_average',0.0),
        'rating_count':p.get('rating_count',0),
        'img':p.get('img')
      })
    print(f'capage={cpage} ({len(all_places)}개 누적)')
    time.sleep(0.2)
  return pd.DataFrame(all_places)
  



In [11]:
# 테스트 시도- 성공

theme_id = 'c9'  # 음식점 테마 코드
rect = '487005,1116407,493715,1122797'  
max_page = 2

df = get_kakao_restaurants(theme_id,rect,max_page=max_page)

print(df.head())

capage=1 (15개 누적)
capage=2 (30개 누적)
    place_id         name                  address       new_address  \
0   20526340     원조조박집 본관       서울 마포구 용강동 40-1 1층  서울 마포구 토정로 313-1   
1   10344284          마포옥      서울 마포구 용강동 50-13 1층    서울 마포구 토정로 312   
2  162078159           모연  서울 마포구 용강동 51-5 영우빌딩 1층    서울 마포구 토정로 318   
3   24529429      프리츠 도화점         서울 마포구 도화동 179-9   서울 마포구 새창로2길 17   
4   27269085  오근내닭갈비 용산본점    서울 용산구 한강로3가 40-90 1층  서울 용산구 이촌로29길 15   

          lon        lat           tel category_depth1 category_depth2  \
0  126.943866  37.539972   02-712-7462             음식점              한식   
1  126.943338  37.539882   02-716-6661             음식점              한식   
2  126.943697  37.539371   02-704-0770             음식점              한식   
3  126.949070  37.541020  02-3275-2045             음식점              카페   
4  126.959709  37.524524   02-797-0131             음식점              한식   

  category_depth3 category_depth4 category_depth5  rating_average  \
0           육류,고기

In [14]:
# 리뷰 크롤링 데이터 구조 확인
all_reviews = []
previous_last_review_id = None
page = 1
strength_maping = {}
limit=10
place_id = '20526340'

url = f'https://place-api.map.kakao.com/places/tab/reviews/kakaomap/{place_id}'

params = {
  'order':'RECOMMENDED',
  'only_photo_review':'false',
  'page': page,
  'limit':limit
  }

if previous_last_review_id:
  params['previous_last_review_id'] = previous_last_review_id

header = {
  'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
    'referer':'https://map.kakao.com/',
    'origin':'https://place.map.kakao.com',
    'pf':'web',
    'priority':'u=1, i',
    'referer':'https://place.map.kakao.com/'
}

resp = requests.get(url,params=params,headers=header)

if resp.status_code !=200:
  print(f'Error {resp.status_code} at page {page}에서 에러가 났습니다.' )

data = resp.json()

data




{'score_set': {'review_count': 227,
  'total_score': 961,
  'average_score': 4.2,
  'strength_counts': [{'id': 5, 'count': 72},
   {'id': 2, 'count': 42},
   {'id': 1, 'count': 29},
   {'id': 3, 'count': 23},
   {'id': 4, 'count': 11}],
  'strength_uv': 75},
 'strength_description': [{'id': 5,
   'name': '맛',
   'icon_url': 'https://t1.kakaocdn.net/kakaomap_mobile/android/icon_review_delicious.png'},
  {'id': 1,
   'name': '가성비',
   'icon_url': 'https://t1.kakaocdn.net/kakaomap_mobile/android/icon_review_money.png'},
  {'id': 2,
   'name': '친절',
   'icon_url': 'https://t1.kakaocdn.net/kakaomap_mobile/android/icon_review_kind.png'},
  {'id': 3,
   'name': '분위기',
   'icon_url': 'https://t1.kakaocdn.net/kakaomap_mobile/android/icon_review_mood.png'},
  {'id': 4,
   'name': '주차',
   'icon_url': 'https://t1.kakaocdn.net/kakaomap_mobile/android/icon_review_parking.png'}],
 'reviews': [{'review_id': 13194108,
   'star_rating': 4,
   'photo_count': 0,
   'status': 'S',
   'strength_ids': [5],


In [1]:
import requests
import time
import pandas as pd

def get_kakao_reviews(place_id, limit=10 ,max_page=100):

  all_reviews = []
  previous_last_review_id = None
  page = 1
  strength_maping = {}

  while True:
    url = f'https://place-api.map.kakao.com/places/tab/reviews/kakaomap/{place_id}'

    params = {
      'order':'RECOMMENDED',
      'only_photo_review':'false',
      'page': page,
      'limit':limit
      }
    
    if previous_last_review_id:
      params['previous_last_review_id'] = previous_last_review_id
    
    header = {
      'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
        'referer':'https://map.kakao.com/',
        'origin':'https://place.map.kakao.com',
        'pf':'web',
        'priority':'u=1, i',
        'referer':'https://place.map.kakao.com/'
    }

    resp = requests.get(url,params=params,headers=header)

    if resp.status_code !=200:
      print(f'Error {resp.status_code} at page {page}에서 에러가 났습니다.' )
      break
    data = resp.json()


    # 강점 키워드 설명은 반복되므로 첫 페이지만 수집한다.
    if page == 1 and 'strength_description' in data:
      strength_maping = {item['id']: item['name'] for item in data['strength_description']}
    reviews = data.get('reviews',[])

    if not reviews:
      break

    for r in reviews:
      all_reviews.append({
        'review_id':r['review_id'],
        'star_rating':r['star_rating'],
        'contents':r.get('contents',''), # r에 'contents' key가 없으면 빈 문자열 '' 반환(default 생략하면 None)
        'registered_at':r.get('registered_at'),
        'strength':[strength_maping.get(ids,'') for ids in r.get('strength_ids', [])]
      })

    if not data.get('has_next',False):
      break

    previous_last_review_id = reviews[-1]['review_id'] #마지막 리뷰의 id로 업데이트
    page += 1

    if page > max_page:
      break
    time.sleep(0.5)
  return pd.DataFrame(all_reviews)


,review_id,star_rating,contents,registered_at,strength
0,13194108,4,,2025-07-11 20:28:12,[맛]
1,13077593,5,맛은 있는데 가격은 갈수록 선 넘어가는 느낌.\n생고기도 아닌 양념 돼갈을 이 가격...,2025-06-25 18:14:57,[]
2,13069895,2,솔직히 위생이 좀 그렇다..,2025-06-24 17:40:54,[]
3,12969993,5,,2025-06-11 00:45:13,[]
4,12929491,5,정말 맛있어요. 고기도 맛있고 동치미국수도 최고 ㅠㅠ,2025-06-06 07:06:12,"[맛, 친절]"
...,...,...,...,...,...
222,678736,5,맛있고 가격 좋고 마지막엔 식혜까지...\n다만 TV 나온 덕분에 기다려야함 ㅠㅠ,2016-04-16 22:53:04,[]
223,720907,4,,2016-03-27 23:24:14,[]
224,845119,4,,2016-03-23 22:45:39,[]
225,890748,4,돼지갈비,2013-08-16 20:02:19,[]


In [25]:
# 테스트 실행 성공
place_id = "20526340"
df = get_kakao_reviews(place_id,limit=20)
df

,review_id,star_rating,contents,registered_at,strength
0,13194108,4,,2025-07-11 20:28:12,[맛]
1,13077593,5,맛은 있는데 가격은 갈수록 선 넘어가는 느낌.\n생고기도 아닌 양념 돼갈을 이 가격...,2025-06-25 18:14:57,[]
2,13069895,2,솔직히 위생이 좀 그렇다..,2025-06-24 17:40:54,[]
3,12969993,5,,2025-06-11 00:45:13,[]
4,12929491,5,정말 맛있어요. 고기도 맛있고 동치미국수도 최고 ㅠㅠ,2025-06-06 07:06:12,"[맛, 친절]"
...,...,...,...,...,...
222,678736,5,맛있고 가격 좋고 마지막엔 식혜까지...\n다만 TV 나온 덕분에 기다려야함 ㅠㅠ,2016-04-16 22:53:04,[]
223,720907,4,,2016-03-27 23:24:14,[]
224,845119,4,,2016-03-23 22:45:39,[]
225,890748,4,돼지갈비,2013-08-16 20:02:19,[]


In [24]:
#데이터 베이스 연결 
user = 'root'
password = 'test1234'
host = 'localhost'
port = 3306
db = 'local_kakao_placeDB'

engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{db}?charset=utf8mb4')